In [1]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# keras imports
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.applications.xception import Xception, preprocess_input
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
#from keras.applications  #import InceptionResNetV2, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
#import cv2
import h5py
import os
import json
import pickle
import datetime
import time

Using TensorFlow backend.


In [2]:
import tensorflow as tf
tf.__version__

'1.2.1'

In [3]:
!jupyter kernelspec list
import sys
sys.executable

Available kernels:
  python3    C:\Users\hj5446\AppData\Local\Continuum\Anaconda3\envs\frac_detect\share\jupyter\kernels\python3


'C:\\Users\\hj5446\\AppData\\Local\\Continuum\\Anaconda3\\envs\\frac_detect\\python.exe'

In [4]:
# filter warnings
# load the user configs
with open('conf/conf_resnet50.json') as f:    #resnet50
    config = json.load(f)
# config variables
model_name = config["model"]
weights = config["weights"]
include_top = config["include_top"]
train_path = config["train_path"]
features_path = config["features_path"]
labels_path = config["labels_path"]
test_size = config["test_size"]
results = config["results"]
model_path = config["model_path"]
#weights_path = config["weights_path"]
print('model_name {}. features_path {}'.format(model_name, features_path))

model_name resnet50. features_path output/flowers_17/resnet50/features.h5


In [5]:
# start time
print("[STATUS] start time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))
start = time.time()

# create the pretrained models
# check for pretrained weight usage or not
# check for top layers to be included or not

if model_name == "vgg16":
    base_model = VGG16(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)
    image_size = (224, 224)
elif model_name == "vgg19":
    base_model = VGG19(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)
    image_size = (224, 224)
elif model_name == "resnet50":
    base_model = ResNet50(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('flatten_1').output) #Flatten
    image_size = (224, 224)
elif model_name == "inceptionv3":
    base_model = InceptionV3(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('avg_pool').output) # this is correct
    #model = Model(input=base_model.input, output=base_model.get_layer('Dense').output)
    image_size = (299, 299)
elif model_name == "xception":
    base_model = Xception(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('avg_pool').output)
    image_size = (299, 299)
elif model_name == "InceptionResNetV2":
    base_model = Xception(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('avg_pool').output)
    image_size = (299, 299)
else:
    base_model = None

print("[INFO] successfully loaded base model and model...")

[STATUS] start time - 2017-11-16 16:08
[INFO] successfully loaded base model and model...


C:\Users\hj5446\AppData\Local\Continuum\Anaconda3\envs\frac_detect\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fl...)`


In [6]:
# path to training dataset
train_files = os.listdir(train_path)

# encode the labels
#print("[INFO] encoding labels...")
#le = LabelEncoder()
#le.fit([tl for tl in train_labels])

# variables to hold features and labels
features = [] # store feature data, x
labels   = [] # store label data, y

label = 0
# loop over all the labels in the folder
for i, file in enumerate(train_files):
    img_path = train_path + "/" + file
    if i %10 == 0:
        print(' {}'.format(i), end=',')
    if (i) % 80 == 0 :
        label = label+1
        print('\n file {}, label {}'.format(file, label))    
    img = image.load_img(img_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    features.append(flat)
    labels.append(label)
print("[INFO] processed - {}".format(i))
print( "[INFO] completed label - {}".format(label))
print(len(labels))

 0,
 file image_0001.jpg, label 1
 10, 20, 30, 40, 50, 60, 70, 80,
 file image_0081.jpg, label 2
 90, 100, 110, 120, 130, 140, 150, 160,
 file image_0161.jpg, label 3
 170, 180, 190, 200, 210, 220, 230, 240,
 file image_0241.jpg, label 4
 250, 260, 270, 280, 290, 300, 310, 320,
 file image_0321.jpg, label 5
 330, 340, 350, 360, 370, 380, 390, 400,
 file image_0401.jpg, label 6
 410, 420, 430, 440, 450, 460, 470, 480,
 file image_0481.jpg, label 7
 490, 500, 510, 520, 530, 540, 550, 560,
 file image_0561.jpg, label 8
 570, 580, 590, 600, 610, 620, 630, 640,
 file image_0641.jpg, label 9
 650, 660, 670, 680, 690, 700, 710, 720,
 file image_0721.jpg, label 10
 730, 740, 750, 760, 770, 780, 790, 800,
 file image_0801.jpg, label 11
 810, 820, 830, 840, 850, 860, 870, 880,
 file image_0881.jpg, label 12
 890, 900, 910, 920, 930, 940, 950, 960,
 file image_0961.jpg, label 13
 970, 980, 990, 1000, 1010, 1020, 1030, 1040,
 file image_1041.jpg, label 14
 1050, 1060, 1070, 1080, 1090, 1100, 1110,

In [7]:
# encode the labels using LabelEncoder
targetNames = np.unique(labels)
#le = LabelEncoder()
#le_labels = le.fit_transform(labels)
le_labels = np.array(labels)
# get the shape of training labels
print("[STATUS] training labels: {}".format(le_labels))
print("[STATUS] training labels shape: {}".format(le_labels.shape))

# save features and labels
h5f_data = h5py.File(features_path, 'w')
h5f_data.create_dataset('dataset_1', data=np.array(features))

h5f_label = h5py.File(labels_path, 'w')
h5f_label.create_dataset('dataset_1', data=np.array(le_labels))

h5f_data.close()
h5f_label.close()

# save model and weights
model_json = model.to_json()
with open(model_path + str(test_size) + ".json", "w") as json_file:
    json_file.write(model_json)

# save weights
model.save_weights(model_path + str(test_size) + ".h5")
print("[STATUS] saved model and weights to disk..")

print("[STATUS] features and labels saved..")

# end time
end = time.time()
print("[STATUS] end time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))

[STATUS] training labels: [ 1  1  1 ..., 17 17 17]
[STATUS] training labels shape: (1360,)
[STATUS] saved model and weights to disk..
[STATUS] features and labels saved..
[STATUS] end time - 2017-11-16 16:14
